In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
from PIL import Image
import io

# Load the dataset from Parquet file
df = pd.read_parquet('train-00000-of-00001-b64601da56687a05.parquet')

# Shorten the dataset to 800 images and corresponding text entries
shortened_df = df.head(800)
noise_dim=100

epochs = 100
batch_size = 32

# Preprocess the images
def preprocess_image(image_bytes):
    image = Image.open(io.BytesIO(image_bytes['bytes']))  # Open image from bytes
    image = image.convert('RGB')  # Convert to RGB format if necessary
    image = image.resize((64, 64))  # Resize image
    image = np.array(image)  # Convert PIL image to numpy array
    image = image.astype(np.float32) / 255.0  # Convert pixel values to [0, 1]
    return image

# Convert image bytes to numpy arrays
images = np.array([preprocess_image(image) for image in shortened_df['image']])

# Define the Generator network
def build_generator(noise_dim):
    input_noise = layers.Input(shape=(noise_dim,))

    # Noise dense layer
    noise_dense = layers.Dense(256)(input_noise)

    # Generator layers
    x = layers.Dense(256 * 8 * 8, activation='relu')(noise_dense)
    x = layers.Reshape((8, 8, 256))(x)
    x = layers.Conv2DTranspose(128, kernel_size=5, strides=2, padding='same', activation='relu')(x)
    x = layers.Conv2DTranspose(64, kernel_size=5, strides=2, padding='same', activation='relu')(x)
    generator_output = layers.Conv2DTranspose(3, kernel_size=5, strides=2, padding='same', activation='sigmoid')(x)

    generator_model = models.Model(inputs=[input_noise], outputs=generator_output)

    return generator_model

# Define the Discriminator network
def build_discriminator():
    input_image = layers.Input(shape=(64, 64, 3))
    # Image embedding layers
    x = layers.Conv2D(64, kernel_size=5, strides=2, padding='same', activation='relu')(input_image)
    x = layers.Conv2D(128, kernel_size=5, strides=2, padding='same', activation='relu')(x)
    image_embedding = layers.Flatten()(x)

    # Discriminator layers
    x = layers.Dense(256, activation='relu')(image_embedding)
    discriminator_output = layers.Dense(1, activation='sigmoid')(x)

    discriminator_model = models.Model(inputs=[input_image], outputs=discriminator_output)

    return discriminator_model

# Define the conditional gan model
def build_gan(generator, discriminator):
    input_noise = layers.Input(shape=(noise_dim,))

    fake_image = generator([input_noise])
    validity = discriminator([fake_image])

    cgan = models.Model(inputs=[input_noise], outputs=validity)

    return cgan

# Instantiate the models
generator = build_generator(noise_dim)
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)

# Compile the models
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
gan.compile(optimizer='adam', loss='binary_crossentropy')

import matplotlib.pyplot as plt

# Train the GAN models
def train_gan(generator, discriminator, gan, images, noise_dim, epochs=10, batch_size=32):
    discriminator_losses = []
    generator_losses = []

    for epoch in range(epochs):
        for batch_start in range(0, len(images), batch_size):
            # Generate noise
            noise = np.random.normal(0, 1, (batch_size, noise_dim))

            # Select a random batch of images and text embeddings
            real_images_batch = images[batch_start:batch_start+batch_size]

            # Generate fake images from noise and text embeddings
            generated_images = generator.predict([noise])

            # Train the Discriminator
            discriminator_loss_real = discriminator.train_on_batch([real_images_batch], np.ones((batch_size, 1)))
            discriminator_loss_fake = discriminator.train_on_batch([generated_images], np.zeros((batch_size, 1)))
            discriminator_loss = 0.5 * np.add(discriminator_loss_real, discriminator_loss_fake)

            # Train the Generator (via cGAN)
            noise = np.random.normal(0, 1, (batch_size, noise_dim))  # Generate new noise for training Generator
            gan_loss = gan.train_on_batch([noise], np.ones((batch_size, 1)))

            # Store losses
            discriminator_losses.append(discriminator_loss[0])
            generator_losses.append(gan_loss)

            # Print training progress
            print(f"Epoch {epoch+1}/{epochs}, Batch {batch_start//batch_size+1}/{len(images)//batch_size}, Discriminator Loss: {discriminator_loss[0]}, Generator Loss: {gan_loss}")

    # Plot Losses
    plt.plot(discriminator_losses, label='Discriminator Loss')
    plt.plot(generator_losses, label='Generator Loss')
    plt.xlabel('Iterations')
    plt.ylabel('Loss')
    plt.title('GAN Training Loss')
    plt.legend()
    plt.savefig('loss_plot.pdf')
    plt.show()

    # Save models
    generator.save('generator_model.h5')
    discriminator.save('discriminator_model.h5')
    gan.save('gan_model.h5')

    # Generate sample images
    sample_noise = np.random.normal(0, 1, (3, noise_dim))
    generated_samples = generator.predict(sample_noise)

    # Plot and save sample images
    fig, axes = plt.subplots(1, 3, figsize=(10, 5))
    for i, ax in enumerate(axes):
        ax.imshow(generated_samples[i])
        ax.axis('off')
    plt.savefig('sample_images.pdf')
    plt.show()

# Train the GAN models
train_gan(generator, discriminator, gan, images, noise_dim, epochs, batch_size)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
Epoch 1/100, Batch 1/25, Discriminator Loss: 3.0802645683288574, Generator Loss: 0.18077336251735687
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Epoch 1/100, Batch 2/25, Discriminator Loss: 4.068397521972656, Generator Loss: 0.24791160225868225
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Epoch 1/100, Batch 3/25, Discriminator Loss: 3.5802724361419678, Generator Loss: 0.324564665555954
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Epoch 1/100, Batch 4/25, Discriminator Loss: 3.0116114616394043, Generator Loss: 0.40517520904541016
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Epoch 1/100, Batch 5/25, Discriminator Loss: 2.577540874481201, Generator Loss: 0.4496529996395111
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Epoch 1/100, Batch 6/25, Discriminator Loss: 2.2656631469726562, Generator Loss: 0.4708159863948822
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Epoch 1/100, Batch 7/25, Discriminator Loss: 2.037034273147583, Generator Loss: 0.49236276745796204
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/s